- DOING
    - 使用`./trial`目录下的txt文件模拟流程（可以按GNSS时间戳判断是否缺数据）
    - 使用提供的**trialname**测试
        - `T62288821d9d01`
        - `T62288821d9d02`

In [4]:
import sys
import requests
import numpy as np
import torch
import time
import copy
import importlib
from parse import parse

In [6]:
sys.path.append('./')
from spd_dnn import *
from data_struct import *
from ftr_generator import *
from coord_utils import *

# 算法流程

## 航向推算

In [3]:
def infer_bearing(sec_gyro_unit_list, crnt_brng, collect_hz=250):
    gy_ys = [u.sen_y for u in sec_gyro_unit_list]
    bearing_diff = np.sum(gy_ys) / collect_hz * -1.0
    return (crnt_brng + bearing_diff) % 360.0

## 速度推算（LSTM）

In [10]:
ftr_hz = 25
collect_hz = 250

model = load_spd_dnn(trained_model='./spd_dnn_weight_ftrHz%d.pt' % ftr_hz,
                     ftr_hz=ftr_hz, collect_hz=collect_hz)
model.eval()

ftr_gen = ftrGenerator(ftr_hz=ftr_hz, collect_hz=collect_hz)

Load trained model: ./spd_dnn_weight_ftrHz25.pt


In [11]:
def calibrate_sample_freq(imu_unit_list, collect_hz=250):
    if len(imu_unit_list) < collect_hz:
        imu_unit_list.extend([copy.deepcopy(imu_unit_list[-1])] * (collect_hz - len(imu_unit_list)))
    elif len(imu_unit_list) > collect_hz:
        imu_unit_list = imu_unit_list[-collect_hz:]
    else:
        pass
    return imu_unit_list


def infer_speed(sec_acc_unit_list, sec_gyro_unit_list, crnt_spd,
                model=None, ftr_gen=None, collect_hz=250):
    sec_acc_unit_list = calibrate_sample_freq(sec_acc_unit_list, collect_hz=collect_hz)
    sec_gyro_unit_lis = calibrate_sample_freq(sec_gyro_unit_list, collect_hz=collect_hz)
    acc_ftr_list = ftr_gen.calc_imu_sec_ftr(sec_acc_unit_list)
    gy_ftr_list = ftr_gen.calc_imu_sec_ftr(sec_gyro_unit_lis)

    acc = torch.tensor([[acc_ftr_list]], dtype=torch.float).permute(0, 2, 1)
    gy = torch.tensor([[gy_ftr_list]], dtype=torch.float).permute(0, 2, 1)
    init_spd = torch.tensor([[[crnt_spd]]], dtype=torch.float)
    pred = model(acc, gy, init_spd=init_spd)
    spd_diff = pred.cpu().detach().squeeze().numpy().tolist()

    return max(0.0, min(25.0, crnt_spd + spd_diff))

## 速度推算（OBD）

In [13]:
def infer_speed_via_obd(obd_unit_list, crnt_spd=None):
    obd_spds = [u.vehicle_speed for u in obd_unit_list]
    spd = np.sum(obd_spds) / len(obd_spds) * 1.0
    return spd

# 服务端配置及请求

- 正式比赛配置

In [6]:
# server = 'https://evaal.aaloa.org/evaalapi/'
# trialname = 'S622team708_02'

# estfmt = "{pts:.3f},{c:.3f},{h:.3f},{s:.3f},{pos}"
# statefmt = "{trialts:.3f},{rem:.3f},{V:.3f},{S:.3f},{p:.3f},{h:.3f},{pts:.3f},{pos}"

- 使用官方服务器测试

In [7]:
# server = 'http://121.196.218.26/evaalapi/'
# trialname = 'T623412f0da401'

# estfmt = "{pts:.3f},{c:.3f},{h:.3f},{s:.3f},{pos}"
# statefmt = "{trialts:.3f},{rem:.3f},{V:.3f},{S:.3f},{p:.3f},{h:.3f},{pts:.3f},{pos}"

- 使用本地服务器测试

In [18]:
server = 'http://127.0.0.1:5000/evaalapi/'
trialname = 'demo2023'

estfmt = "{pts:.3f},{c:.3f},{h:.3f},{s:.3f},{pos}"
statefmt = "{trialts:.3f},{rem:.3f},{V:.3f},{S:.3f},{p:.3f},{h:.3f},{pts:.3f},{pos}"

In [19]:
def do_state():
    r = requests.get(server + trialname + '/state')
    print(r.text)
    s = parse(statefmt, r.text)
    return s.named
    
    
def do_reload():
    r = requests.get(server + trialname + '/reload')
    s = parse(statefmt, r.text)
    print(s.named)

In [20]:
def parse_response(text):
    sen_str_list = text.splitlines()
    acc_unit_list, gy_unit_list = [], []
    obd_unit_list = []
    gnss_unit = None
    for sen_str in sen_str_list:
        if sen_str.startswith('ACCE'):
            acc_unit_list.append(acceUnit(sen_str))
        elif sen_str.startswith('GYRO'):
            gy_unit_list.append(gyroUnit(sen_str))
        elif sen_str.startswith('GNSS'):
            gnss_unit = gnssUnit(sen_str)
        elif sen_str.startswith('OBD'):
            obd_unit_list.append(obdUnit(sen_str))
        else:
            continue
    return acc_unit_list, gy_unit_list, obd_unit_list, gnss_unit


def estimate_next_loc(acc_unit_list, gy_unit_list, obd_unit_list, gnss_unit, 
                      model=None, ftr_gen=None, ftr_hz=50, collect_hz=250):
    crnt_brng = gnss_unit.bearing
    crnt_spd = gnss_unit.loc_speed
    if crnt_spd < 0:
        crnt_spd = gnss_unit.speed
    
    next_brng = infer_bearing(gy_unit_list, crnt_brng, collect_hz=collect_hz)
    
    if len(obd_unit_list) > 10:
        next_spd = infer_speed_via_obd(obd_unit_list)
    else:
        next_spd = infer_speed(acc_unit_list, gy_unit_list, crnt_spd, 
                               model=model, ftr_gen=ftr_gen, collect_hz=collect_hz)

    next_lng, next_lat = geo_util.position_from_angle_and_dist(gnss_unit.lng, gnss_unit.lat, 
                                                               next_brng, next_spd)
    return next_lng, next_lat, next_spd, next_brng


def estimate_next_loc_by_pred(acc_unit_list, gy_unit_list, obd_unit_list, pred_loc, 
                              model=None, ftr_gen=None, ftr_hz=25, collect_hz=250):
    crnt_lng, crnt_lat, crnt_spd, crnt_brng = pred_loc
    
    next_brng = infer_bearing(gy_unit_list, crnt_brng, collect_hz=collect_hz)
    
    if len(obd_unit_list) > 10:
        next_spd = infer_speed_via_obd(obd_unit_list)
    else:
        next_spd = infer_speed(acc_unit_list, gy_unit_list, crnt_spd, 
                               model=model, ftr_gen=ftr_gen, collect_hz=collect_hz)
    
    next_lng, next_lat = geo_util.position_from_angle_and_dist(crnt_lng, crnt_lat, 
                                                               next_brng, next_spd)
    return next_lng, next_lat, next_spd, next_brng
    
    
def estimate_next_loc_by_dft(crnt_lng, crnt_lat, crnt_spd, crnt_brng):
    next_brng, next_spd = crnt_brng, crnt_spd
    next_lng, next_lat = geo_util.position_from_angle_and_dist(crnt_lng, crnt_lat, 
                                                               next_brng, next_spd)
    return next_lng, next_lat, next_spd, next_brng
    

In [21]:
# do_reload()

In [22]:
# s = do_state()
# print(s)

In [23]:
# r = requests.get(server + trialname + '/estimates')
# print(r.text)

In [24]:
# r = requests.get(server + trialname + '/nextdata?horizon=1.0&position=0,0,0')
# print(r.text)

# r = requests.get(server + trialname + '/log')
# print(r.text)

In [25]:
init_pos = None  # 车辆初始位置
altitude = 0.0  # 高程
pre_gnss = None  # 上一时刻GNSS
pred_loc = None
next_position = None
gt_gnss_unit_list = []

sec_cnt = 0
request_arr = []

do_reload()  # 正式比赛需要注释
while True:
    s = do_state()
    if s['trialts'] == 0.0:
        print('testing start...')
        try:
            lat, lng, altitude = [float(x) for x in s['pos'].split(',')]
        except ValueError as e:
            lat, lng, altitude = [float(x) for x in s['pos'].split(';')]
        init_pos = [lng, lat]
    elif s['trialts'] < 0:
        print('testing timeout...')
        break
    else:
        pass
    
    if next_position is None:
        r = requests.get(server + trialname + '/nextdata?horizon=1.0')
    else:
        r = requests.get(server + trialname + \
                         '/nextdata?horizon=1.0&position=%s,%.1f' % (next_position, altitude))
    
    acc_unit_list, gy_unit_list, obd_unit_list, gnss_unit = parse_response(r.text)
    
#     request_arr.append(r.text.splitlines())
    if gnss_unit is None:
        print(len(acc_unit_list))
        print(len(gy_unit_list))
        
    if gnss_unit is not None:
        gt_gnss_unit_list.append(gnss_unit)
        
    if len(acc_unit_list) < 10 and gnss_unit is None:
        if pre_gnss is not None:
            lng, lat, spd, brng = pre_gnss.lng, pre_gnss.lat, pre_gnss.loc_speed, pre_gnss.bearing
            if spd < 0:
                spd = pre_gnss.speed
            next_lng, next_lat, \
            next_spd, next_brng = estimate_next_loc_by_dft(lng, lat, spd, brng)
            next_position = '%.6f,%.6f' % (next_lat, next_lng)
            pred_loc = [next_lng, next_lat, next_spd, next_brng]
        elif pred_loc is not None:
            next_lng, next_lat, \
            next_spd, next_brng = estimate_next_loc_by_dft(pred_loc[0], pred_loc[1], 
                                                           pred_loc[2], pred_loc[3])
            next_position = '%.6f,%.6f' % (next_lat, next_lng)
            pred_loc = [next_lng, next_lat, next_spd, next_brng]
        else:
            pass
        time.sleep(0.501)
        sec_cnt += 1
        continue
    
    if gnss_unit is not None:
        gnss_unit.pre_gnss = pre_gnss
        pre_gnss = gnss_unit
        next_position = '%.6f,%.6f' % (gnss_unit.lat, gnss_unit.lng)
    elif pre_gnss is not None:
        next_lng, next_lat, \
        next_spd, next_brng = estimate_next_loc(acc_unit_list, gy_unit_list, obd_unit_list, pre_gnss, 
                                                model=model, ftr_gen=ftr_gen, 
                                                ftr_hz=ftr_hz, collect_hz=collect_hz)
        next_position = '%.6f,%.6f' % (next_lat, next_lng)
        pred_loc = [next_lng, next_lat, next_spd, next_brng]
        pre_gnss = None
    elif pred_loc is not None:
        next_lng, next_lat, \
        next_spd, next_brng = estimate_next_loc_by_pred(acc_unit_list, gy_unit_list, obd_unit_list, pred_loc, 
                                                        model=model, ftr_gen=ftr_gen, 
                                                        ftr_hz=ftr_hz, collect_hz=collect_hz)
        next_position = '%.6f,%.6f' % (next_lat, next_lng)
        pred_loc = [next_lng, next_lat, next_spd, next_brng]
    else:
        pass
        
    time.sleep(0.101)
    sec_cnt += 1
#     if sec_cnt == 2500:
#         break

{'trialts': 0.0, 'rem': -1.0, 'V': 3.0, 'S': 15.0, 'p': 0.0, 'h': 0.0, 'pts': 0.0, 'pos': '40.002208,116.385118,85.000'}
0.000,-1.000,3.000,15.000,0.000,0.000,0.000,40.002208,116.385118,85.000
testing start...
35.680,17.866,3.000,15.000,1694328571.119,1.000,34.680,40.002208,116.385118,85.000
36.680,17.852,3.000,15.000,1694328571.261,1.000,35.680,40.002201,116.385107,85.0
37.680,17.854,3.000,15.000,1694328571.417,1.000,36.680,40.002200,116.385109,85.0
38.680,17.854,3.000,15.000,1694328571.571,1.000,37.680,40.002203,116.385114,85.0
39.680,17.853,3.000,15.000,1694328571.724,1.000,38.680,40.002208,116.385118,85.0
40.680,17.852,3.000,15.000,1694328571.876,1.000,39.680,40.002211,116.385127,85.0
41.680,17.851,3.000,15.000,1694328572.036,1.000,40.680,40.002213,116.385134,85.0
42.680,17.845,3.000,15.000,1694328572.194,1.000,41.680,40.002213,116.385135,85.0
43.680,17.851,3.000,15.000,1694328572.359,1.000,42.680,40.002212,116.385136,85.0
44.680,17.846,3.000,15.000,1694328572.514,1.000,43.680,40.0

134.680,17.843,3.000,15.000,1694328587.009,1.000,133.680,40.003474,116.386168,85.0
135.680,17.837,3.000,15.000,1694328587.169,1.000,134.680,40.003478,116.386232,85.0
136.680,17.809,3.000,15.000,1694328587.336,1.000,135.680,40.003482,116.386296,85.0
137.680,17.867,3.000,15.000,1694328587.531,1.000,136.680,40.003484,116.386358,85.0
138.680,17.850,3.000,15.000,1694328587.666,1.000,137.680,40.003486,116.386410,85.0
139.680,17.846,3.000,15.000,1694328587.820,1.000,138.680,40.003488,116.386458,85.0
140.680,17.849,3.000,15.000,1694328587.976,1.000,139.680,40.003489,116.386506,85.0
141.680,17.849,3.000,15.000,1694328588.131,1.000,140.680,40.003492,116.386555,85.0
142.680,17.849,3.000,15.000,1694328588.285,1.000,141.680,40.003496,116.386603,85.0
143.680,17.838,3.000,15.000,1694328588.440,1.000,142.680,40.003499,116.386656,85.0
144.680,17.810,3.000,15.000,1694328588.605,1.000,143.680,40.003502,116.386709,85.0
145.680,17.854,3.000,15.000,1694328588.800,1.000,144.680,40.003505,116.386764,85.0
146.

234.680,17.842,3.000,15.000,1694328603.254,1.000,233.680,40.001699,116.390307,85.0
235.680,17.841,3.000,15.000,1694328603.415,1.000,234.680,40.001610,116.390305,85.0
236.680,17.843,3.000,15.000,1694328603.576,1.000,235.680,40.001525,116.390303,85.0
237.680,17.803,3.000,15.000,1694328603.736,1.000,236.680,40.001450,116.390299,85.0
238.680,17.867,3.000,15.000,1694328603.937,1.000,237.680,40.001388,116.390295,85.0
239.680,17.850,3.000,15.000,1694328604.077,1.000,238.680,40.001341,116.390292,85.0
240.680,17.839,3.000,15.000,1694328604.230,1.000,239.680,40.001304,116.390292,85.0
241.680,17.836,3.000,15.000,1694328604.394,1.000,240.680,40.001277,116.390287,85.0
242.680,17.842,3.000,15.000,1694328604.562,1.000,241.680,40.001258,116.390285,85.0
243.680,17.824,3.000,15.000,1694328604.722,1.000,242.680,40.001243,116.390288,85.0
244.680,17.837,3.000,15.000,1694328604.901,1.000,243.680,40.001236,116.390290,85.0
245.680,17.847,3.000,15.000,1694328605.066,1.000,244.680,40.001235,116.390294,85.0
246.

334.680,17.803,3.000,15.000,1694328619.605,1.000,333.680,39.997797,116.390788,85.0
335.680,17.855,3.000,15.000,1694328619.807,1.000,334.680,39.997704,116.390810,85.0
336.680,17.855,3.000,15.000,1694328619.954,1.000,335.680,39.997611,116.390828,85.0
337.680,17.839,3.000,15.000,1694328620.102,1.000,336.680,39.997518,116.390842,85.0
338.680,17.846,3.000,15.000,1694328620.267,1.000,337.680,39.997426,116.390854,85.0
339.680,17.843,3.000,15.000,1694328620.423,1.000,338.680,39.997338,116.390865,85.0
340.680,17.837,3.000,15.000,1694328620.584,1.000,339.680,39.997257,116.390877,85.0
341.680,17.829,3.000,15.000,1694328620.751,1.000,340.680,39.997187,116.390886,85.0
342.680,17.822,3.000,15.000,1694328620.925,1.000,341.680,39.997128,116.390894,85.0
343.680,17.863,3.000,15.000,1694328621.108,1.000,342.680,39.997076,116.390906,85.0
344.680,17.847,3.000,15.000,1694328621.247,1.000,343.680,39.997030,116.390920,85.0
345.680,17.831,3.000,15.000,1694328621.404,1.000,344.680,39.996987,116.390932,85.0
346.

434.680,17.844,3.000,15.000,1694328635.871,1.000,433.680,39.996689,116.387302,85.0
435.680,17.846,3.000,15.000,1694328636.030,1.000,434.680,39.996687,116.387271,85.0
436.680,17.842,3.000,15.000,1694328636.187,1.000,435.680,39.996683,116.387228,85.0
437.680,17.842,3.000,15.000,1694328636.349,1.000,436.680,39.996681,116.387171,85.0
438.680,17.833,3.000,15.000,1694328636.510,1.000,437.680,39.996678,116.387104,85.0
439.680,17.837,3.000,15.000,1694328636.681,1.000,438.680,39.996673,116.387029,85.0
440.680,17.847,3.000,15.000,1694328636.847,1.000,439.680,39.996670,116.386946,85.0
441.680,17.841,3.000,15.000,1694328637.003,1.000,440.680,39.996665,116.386858,85.0
442.680,17.837,3.000,15.000,1694328637.165,1.000,441.680,39.996661,116.386766,85.0
443.680,17.847,3.000,15.000,1694328637.331,1.000,442.680,39.996658,116.386666,85.0
444.680,17.830,3.000,15.000,1694328637.488,1.000,443.680,39.996657,116.386564,85.0
445.680,17.836,3.000,15.000,1694328637.662,1.000,444.680,39.996655,116.386460,85.0
446.

...

5089.680,17.831,3.000,15.000,1694329378.873,1.000,5088.680,39.996459,116.382895,85.0
5090.680,17.845,3.000,15.000,1694329379.045,1.000,5089.680,39.996467,116.383053,85.0
5091.680,17.838,3.000,15.000,1694329379.202,1.000,5090.680,39.996475,116.383213,85.0
5092.680,17.858,3.000,15.000,1694329379.367,1.000,5091.680,39.996487,116.383375,85.0
5093.680,17.841,3.000,15.000,1694329379.511,1.000,5092.680,39.996498,116.383544,85.0
5094.680,17.858,3.000,15.000,1694329379.672,1.000,5093.680,39.996507,116.383715,85.0
5095.680,17.858,3.000,15.000,1694329379.817,1.000,5094.680,39.996514,116.383885,85.0
5096.680,17.853,3.000,15.000,1694329379.963,1.000,5095.680,39.996516,116.384051,85.0
5097.680,17.835,3.000,15.000,1694329380.117,1.000,5096.680,39.996519,116.384208,85.0
5098.680,17.826,3.000,15.000,1694329380.298,1.000,5097.680,39.996518,116.384350,85.0
5099.680,17.810,3.000,15.000,1694329380.475,1.000,5098.680,39.996518,116.384479,85.0
5100.680,17.858,3.000,15.000,1694329380.669,1.000,5099.680,39.996

5187.680,17.842,3.000,15.000,1694329394.539,1.000,5186.680,39.996592,116.387097,85.0
5188.680,17.842,3.000,15.000,1694329394.701,1.000,5187.680,39.996594,116.387171,85.0
5189.680,17.848,3.000,15.000,1694329394.862,1.000,5188.680,39.996597,116.387253,85.0
5190.680,17.843,3.000,15.000,1694329395.017,1.000,5189.680,39.996599,116.387343,85.0
5191.680,17.845,3.000,15.000,1694329395.177,1.000,5190.680,39.996601,116.387438,85.0
5192.680,17.845,3.000,15.000,1694329395.334,1.000,5191.680,39.996603,116.387536,85.0
5193.680,17.839,3.000,15.000,1694329395.493,1.000,5192.680,39.996605,116.387640,85.0
5194.680,17.843,3.000,15.000,1694329395.657,1.000,5193.680,39.996608,116.387750,85.0
5195.680,17.854,3.000,15.000,1694329395.816,1.000,5194.680,39.996611,116.387866,85.0
5196.680,17.802,3.000,15.000,1694329395.965,1.000,5195.680,39.996614,116.387989,85.0
5197.680,17.868,3.000,15.000,1694329396.168,1.000,5196.680,39.996619,116.388118,85.0
5198.680,17.846,3.000,15.000,1694329396.302,1.000,5197.680,39.996

5285.680,17.844,3.000,15.000,1694329410.194,1.000,5284.680,39.996696,116.388917,85.0
5286.680,17.844,3.000,15.000,1694329410.352,1.000,5285.680,39.996697,116.388844,85.0
5287.680,17.855,3.000,15.000,1694329410.510,1.000,5286.680,39.996695,116.388761,85.0
5288.680,17.836,3.000,15.000,1694329410.658,1.000,5287.680,39.996689,116.388673,85.0
5289.680,17.847,3.000,15.000,1694329410.824,1.000,5288.680,39.996683,116.388581,85.0
5290.680,17.823,3.000,15.000,1694329410.980,1.000,5289.680,39.996680,116.388482,85.0
5291.680,17.862,3.000,15.000,1694329411.160,1.000,5290.680,39.996676,116.388376,85.0
5292.680,17.855,3.000,15.000,1694329411.301,1.000,5291.680,39.996672,116.388259,85.0
5293.680,17.859,3.000,15.000,1694329411.449,1.000,5292.680,39.996668,116.388136,85.0
5294.680,17.855,3.000,15.000,1694329411.591,1.000,5293.680,39.996661,116.388015,85.0
5295.680,17.840,3.000,15.000,1694329411.740,1.000,5294.680,39.996654,116.387896,85.0
5296.680,17.834,3.000,15.000,1694329411.902,1.000,5295.680,39.996

5383.680,17.863,3.000,15.000,1694329425.789,1.000,5382.680,39.996602,116.385541,85.0
5384.680,17.836,3.000,15.000,1694329425.929,1.000,5383.680,39.996609,116.385507,85.0
5385.680,17.834,3.000,15.000,1694329426.096,1.000,5384.680,39.996619,116.385474,85.0
5386.680,17.858,3.000,15.000,1694329426.264,1.000,5385.680,39.996632,116.385448,85.0
5387.680,17.807,3.000,15.000,1694329426.410,1.000,5386.680,39.996646,116.385425,85.0
5388.680,17.865,3.000,15.000,1694329426.607,1.000,5387.680,39.996655,116.385417,85.0
5389.680,17.844,3.000,15.000,1694329426.744,1.000,5388.680,39.996662,116.385410,85.0
5390.680,17.846,3.000,15.000,1694329426.904,1.000,5389.680,39.996671,116.385408,85.0
5391.680,17.844,3.000,15.000,1694329427.061,1.000,5390.680,39.996689,116.385410,85.0
5392.680,17.847,3.000,15.000,1694329427.219,1.000,5391.680,39.996725,116.385402,85.0
5393.680,17.839,3.000,15.000,1694329427.375,1.000,5392.680,39.996772,116.385408,85.0
5394.680,17.846,3.000,15.000,1694329427.539,1.000,5393.680,39.996

5481.680,17.848,3.000,15.000,1694329441.204,1.000,5480.680,40.000482,116.385198,85.0
5482.680,17.839,3.000,15.000,1694329441.360,1.000,5481.680,40.000482,116.385198,85.0
5483.680,17.839,3.000,15.000,1694329441.525,1.000,5482.680,40.000483,116.385198,85.0
5484.680,17.837,3.000,15.000,1694329441.689,1.000,5483.680,40.000483,116.385198,85.0
5485.680,17.826,3.000,15.000,1694329441.857,1.000,5484.680,40.000483,116.385198,85.0
5486.680,17.832,3.000,15.000,1694329442.034,1.000,5485.680,40.000483,116.385198,85.0
5487.680,17.851,3.000,15.000,1694329442.208,1.000,5486.680,40.000479,116.385197,85.0
5488.680,17.850,3.000,15.000,1694329442.361,1.000,5487.680,40.000479,116.385195,85.0
5489.680,17.840,3.000,15.000,1694329442.513,1.000,5488.680,40.000485,116.385192,85.0
5490.680,17.847,3.000,15.000,1694329442.675,1.000,5489.680,40.000498,116.385188,85.0
5491.680,17.843,3.000,15.000,1694329442.831,1.000,5490.680,40.000522,116.385183,85.0
5492.680,17.844,3.000,15.000,1694329442.996,1.000,5491.680,40.000

In [26]:
est_fd = open('./ScoringResult01.csv', 'w')

r = requests.get(server + trialname + '/estimates')
for i, line in enumerate(r.text.splitlines()):
    if i in {0, 1}:
        continue
    s = parse(estfmt, line)
    ts = int(s.named['pts'])
    lat, lng, altitude = [float(x) for x in s.named['pos'].split(',')]
    est_fd.write('%d,%.6f,%.6f,%.1f\n' % (ts, lat, lng, altitude))

In [27]:
r = requests.get(server + trialname + '/estimates')
print(len(r.text.splitlines()))

5497


In [28]:
print(parse(estfmt, r.text.splitlines()[-1]).named)

{'pts': 5529.68, 'c': 1694329449.486, 'h': 1.0, 's': 15.0, 'pos': '40.002315,116.385128,85.0'}


## 评估

In [29]:
ts_estimate_loc_map = {}

r = requests.get(server + trialname + '/estimates')
for i, line in enumerate(r.text.splitlines()):
    if i in {0, 1}:
        continue
    s = parse(estfmt, line)
    ts = int(float(s.named['pts']))
    loc_str = s.named['pos']
    lat, lng = float(loc_str.split(',')[0]), float(loc_str.split(',')[1])
    lng, lat = geo_util.wgs84_to_gcj02(lng, lat)
    ts_estimate_loc_map[ts] = [lng, lat]
    
ts_list = sorted(ts_estimate_loc_map.keys(), reverse=False)
with open('offline_estimate.csv', 'w') as fd:
    fd.write('ts,lng,lat\n')
    for i, ts in enumerate(ts_list):
        lng, lat = ts_estimate_loc_map[ts]
        if i > 0:
            ts = ts - ts_list[0]
        fd.write('%d,%.6f,%.6f\n' % (ts, lng, lat))

In [30]:
ts_gnss_loc_map = {}

for i, gnss in enumerate(gt_gnss_unit_list):
    ts = int(round(gnss.app_ts, 0))
    lng, lat = gnss.lng_gcj02, gnss.lat_gcj02
    ts_gnss_loc_map[ts] = [lng, lat]
    
ts_list = sorted(ts_gnss_loc_map.keys(), reverse=False)
with open('offline_gnss.csv', 'w') as fd:
    fd.write('ts,lng,lat\n')
    for i, ts in enumerate(ts_list):
        lng, lat = ts_gnss_loc_map[ts]
        if i > 0:
            ts = ts - ts_list[0]
        fd.write('%d,%.6f,%.6f\n' % (ts, lng, lat))

In [31]:
len(ts_gnss_loc_map)

2968

In [32]:
dist_err_list = []

for ts, (lng, lat) in ts_estimate_loc_map.items():
    if ts not in ts_gnss_loc_map and ts + 1 in ts_gnss_loc_map:
        dist_err = geo_util.distance(lng, lat, 
                                     ts_gnss_loc_map[ts + 1][0], ts_gnss_loc_map[ts + 1][1])
        dist_err_list.append(dist_err)

In [33]:
print(np.percentile(dist_err_list, 25))
print(np.percentile(dist_err_list, 50))
print(np.percentile(dist_err_list, 75))

65.4118115
88.153717
93.9171915


In [34]:
dist_err_list

[88.153717, 99.680666, 42.669906]